In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from scipy.stats.stats import pearsonr  
%matplotlib inline

In [2]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn import svm
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import catboost
from mlxtend.classifier import StackingCVClassifier,StackingClassifier
from sklearn import model_selection

In [3]:
test_com=pd.read_csv(r"F:\data mining\projects\tianchi\happiness\happiness_test_complete.csv",parse_dates=['survey_time'],encoding='latin-1')
test_abbr=pd.read_csv(r"F:\data mining\projects\tianchi\happiness\happiness_test_abbr.csv",parse_dates=['survey_time'], encoding='latin-1')
train_com=pd.read_csv(r"F:\data mining\projects\tianchi\happiness\happiness_train_complete.csv",parse_dates=['survey_time'])
train_abbr=pd.read_csv(r"F:\data mining\projects\tianchi\happiness\happiness_train_abbr.csv",parse_dates=['survey_time'])

In [4]:
train_com = train_com.loc[train_com['happiness'] != -8]
y=train_com.pop('happiness')
df_all=pd.concat((train_com,test_com),axis=0)

# train_abbr = train_abbr.loc[train_abbr['happiness'] != -8]
# y=train_abbr.pop('happiness')
# df_all=pd.concat((train_abbr,test_abbr),axis=0)

In [5]:

quantile_income1=train_abbr[(train_abbr.income>0)&(train_abbr.survey_type==1)].income.quantile(np.arange(0,1,0.1)).values
quantile_income2=train_abbr[(train_abbr.income>0)&(train_abbr.survey_type==2)].income.quantile(np.arange(0,1,0.1)).values
quantile_family_income1=train_abbr[(train_abbr.family_income>0)&(train_abbr.survey_type==1)].family_income.quantile(np.arange(0,1,0.1)).values
quantile_family_income2=train_abbr[(train_abbr.family_income>0)&(train_abbr.survey_type==2)].family_income.quantile(np.arange(0,1,0.1)).values

In [6]:
def quantile_func(quantile,e):

    if e>0 and e<=quantile[1]:
        re=1
    elif e>quantile[1] and e<=quantile[2]:
        re= 2
    elif e>quantile[2] and e<=quantile[3]:
        re= 3
    elif e>quantile[3] and e<=quantile[4]:
        re= 4
    elif e>quantile[4] and e<=quantile[5]:
        re= 5
    elif e>quantile[5] and e<=quantile[6]:
        re= 6
    elif e>quantile[6] and e<=quantile[7]:
        re= 7
    elif e>quantile[7] and e<=quantile[8]:
        re= 8
    elif e>quantile[8] and e<=quantile[9]:
        re= 9
    elif e>quantile[9]:
        re= 10
    else : #e<=0 , or e==nan
        re=e
    return re
def level_income(x,quantile1,quantile2):
    e=x[1]
    if x[0]==1:
        return quantile_func(quantile1,e)
    else:
        return quantile_func(quantile2,e)

In [7]:
# def level_income(df,col):
   
#     x2=pd.qcut(df[(df[col]>0)&(df.survey_type==2)].income,10,labels=range(1,11))
#     x1=pd.qcut(df[(df[col]>0)&(df.survey_type==1)].income,10,labels=range(1,11))
#     x0=df[(df[col]<=0)][col].astype(np.int64)
#     return pd.concat([x0,x1,x2])

def log_income(e):

    if e> 0:
        return np.log10(e)//1
    else:
        return e
def log_floor_area(e):
    if e> 0:
        return np.log(e)//1
    else:
        return e
def level_BMI(e):
    if e <15:
        r = 0
    elif e > 15 and e <= 16:
        r=1
    elif e >16 and e <= 18.5:
        r=2
    elif e > 18.5 and e <= 25:
        r=3
    elif e > 25 and e <= 30:
        r=4
    elif e > 30 and e <= 40:
        r=5
    else:
        r = 6
    return r
def level_house(e):
    if e<=4:
        return e
    elif e >= 5 and e <= 10:
        return 5
    else:
        return 6
def level_work_yr(e):
    if e<=1:
        return e
    elif e > 1 and e <= 5:
        return 2
    elif e > 5 and e <= 10:
        return 3
    elif e > 10 and e <= 20:
        return 4
    elif e > 20:
        return 5
    else:#nan
        return -2 #dont know
def level_family_m(e):
    if e<=3:
        return e
    elif e > 3 and e <= 5:
        return 4
    elif e > 5 and e <= 10:
        return 5
    else:
        return 6
def dfprocess(df):
    df['BMI']=(df.weight_jin/2)/(df.height_cm/100)**2
    df['BMI_level']=df['BMI'].apply(level_BMI)
    df['BMI']=df['BMI']//1
    df['age']=(2015-df['birth'])
    df['age-level']=df['age']//10
    df["house"]=df['house'].apply(level_house)
    df["family_m_level"]=df['family_m'].apply(level_family_m)
    df["work_yr_level"]=df['work_yr'].apply(level_work_yr)
#     df["family_income_log"]=df['family_income'].apply(log_income)
    df["floor_area_log"]=df['floor_area'].apply(log_floor_area)
    df["income_level"]=df[['survey_type','income']].apply(lambda y: level_income(y,quantile_income1,quantile_income2 ),axis=1)
    df["family_income_level"]=df[['survey_type','family_income']].apply(lambda y: level_income(y,quantile_family_income1,quantile_family_income2 ),axis=1)

    
#     df = df.loc[df['happiness'] != -8]
    df=df.drop(['id','survey_time','age'],axis=1)
    return df

In [8]:
def calc_mean(*predict_test_y):
    mean=np.mean(np.column_stack((predict_test_y)),axis=1)
    return np.rint(mean)

In [9]:
df_all['family_income'].fillna(df_all['family_income'].mean(),inplace=True)
df_all=dfprocess(df_all)

In [10]:
def level_income_diff(e):
    if e<=4:
        return e
    elif e >= 5 and e <= 10:
        return 5
    elif e>10:
        return 6
    else:
        return -1

def level_by10(e):
    if e>0:
        re=e//10 +1
    elif e<=0:
        re=e
    else: #nan
        re=0
    return re
def decade_level(e):
    
    if e>0 and e<2100:
        e=2015-e
        re=e//10+1

    elif e<=0:
        re=e
    else: #nan, 9997
        re=0
    return re
def dfProcessCom(df):
    # difference between actual income and expectation
    train_com['income_diff']=train_com.apply(lambda x: np.ceil((x['inc_exp']-x['income'])/x['income']) if ((x['inc_exp']>0) &(x['income']>0) )else np.nan, axis=1)
    train_com['income_diff']=train_com['income_diff'].apply(level_income_diff)
        
    cols=['public_service_1','public_service_2','public_service_3','public_service_4','public_service_5','public_service_6','public_service_7','public_service_8','public_service_9',]
    for col in cols:
        df[col]=df[col].apply(level_by10)
    df['marital_years']=df['marital_now'].apply(lambda x: np.log2(2015-x)//1 if x>0 else x)
    df['inc_exp']=df['inc_exp'].apply(log_income)
    df['s_income']=df['s_income'].apply(log_income)
    cols=['m_birth','f_birth','marital_1st','edu_yr','join_party','marital_now','s_birth']
    for col in cols:
        df[col]=df[col].apply(decade_level)
#     df['floor_area']=df['floor_area'].apply(np.log1p).round(1)
    return df 

In [11]:
df_all=dfProcessCom(df_all)

In [12]:
df_all['work_type'].fillna(0,inplace=True)
df_all['work_status'].fillna(0,inplace=True)
df_all['work_manage'].fillna(0,inplace=True)
df_all['work_yr'].fillna(0,inplace=True)
df_all['minor_child'].fillna(0,inplace=True)
df_all['marital_years'].fillna(0,inplace=True)
df_all['edu_status'].fillna(0,inplace=True)
df_all['hukou_loc'].fillna(4,inplace=True)
df_all['social_neighbor'].fillna(-8,inplace=True)
df_all['social_friend'].fillna(-8,inplace=True)
df_all['family_income'].fillna(df_all['family_income'].mean(),inplace=True)
df_all=df_all.drop(['edu_other', 'invest_other', 'property_other', 's_work_status', 's_work_type',    's_birth',
                    's_work_exper', 's_income','s_hukou','s_political', 's_edu',
                   'invest_0', 'invest_2', 'invest_3', 'invest_4', 'invest_5', 'invest_6','invest_7', 'invest_8'],axis=1)

In [13]:
# df_all.describe().T.iloc[:50]
# df_all.describe().T.iloc[50:100]
# df_all.describe().T.iloc[100:]

In [14]:
cols_large=['birth', 'income', 'floor_area' ,'height_cm', 'weight_jin', 'family_income', 's_birth']

In [15]:
cols_all=df_all.columns.values
cols_old=['birth','floor_area', 'height_cm', 'weight_jin', 'work_yr', 'income','family_income',]
cols_new=['BMI', 'BMI_level', 'age-level', 'floor_area_log', 'income_level', 'family_income_level','work_yr_level','family_income_level'],
cols_abbr=[ 'survey_type', 'province', 'city', 'county', 'gender',
       'birth', 'nationality', 'religion', 'religion_freq', 'edu', 'income',
       'political', 'floor_area', 'height_cm', 'weight_jin', 'health',
       'health_problem', 'depression', 'hukou', 'socialize', 'relax', 'learn',
       'equity', 'class', 'work_exper', 'work_status', 'work_yr', 'work_type',
       'work_manage', 'family_income', 'family_m', 'family_status', 'house',
       'car', 'marital', 'status_peer', 'status_3_before', 'view',
       'inc_ability']
float_cols=['birth', 'income',   'floor_area', 'height_cm', 'weight_jin', 'work_yr', 'family_income','BMI','family_m',]
cat_cols_abbr=['survey_type', 'province', 'city', 'county', 'gender', 'nationality', 'religion', 'political',  'hukou','work_exper', 'work_status',
       'work_type', 'work_manage',  'marital',]
cat_cols_com=['survey_type', 'province', 'city', 'county', 'gender', 'nationality', 'religion','edu_status' ,'political',  'hukou','work_exper', 'work_status',
       'work_type', 'work_manage',  'marital','hukou_loc','s_political','s_hukou','s_work_exper','s_work_status','f_political','f_work_14','m_birth','m_political','m_work_14']

In [16]:
dummy_cols=np.intersect1d(cols_all,cat_cols_com)
df_dummies=pd.get_dummies(data=df_all, columns=dummy_cols)

In [17]:
X_full=df_dummies.iloc[:len(y)];y_full=y

In [20]:
testindex=np.loadtxt(r'testindex.txt')
trainindex=np.loadtxt(r'trainindex.txt')

In [21]:
X_train, X_test = df_dummies.iloc[:len(y)].loc[trainindex],df_dummies.iloc[:len(y)].loc[testindex],

y_train, y_test = y.loc[trainindex],y.loc[testindex],

In [22]:
test_submit=df_dummies.iloc[len(y):]

In [23]:

cols=df_dummies.columns
cols_cat=np.intersect1d(cols,cat_cols_com).tolist()
cols_cat_cat=np.setdiff1d(cols_cat,np.array(['work_status', 'work_yr', 'work_type', 'work_manage'])).tolist()

In [24]:
len(cols)

483

In [27]:
lgb_r1 = lgb.LGBMRegressor(boosting_type='gbdt', colsample_bytree=1,learning_rate=0.05,n_estimators=1000,max_depth=4)
lgb_r1.fit(X_train[cols], y_train,eval_metric ='mean_squared_error')
lgb_r1_y2 = lgb_r1.predict(X_test[cols],)
mse=mean_squared_error(np.rint(lgb_r1_y2), y_test)
print(mse)
mean_squared_error(lgb_r1_y2, y_test)

0.513770655983976


0.45466843403949836

In [26]:
xgb_r1 = xgb.XGBRegressor(max_depth=5)
xgb_r1.fit(X_train[cols], y_train)
xgb_r1_y3 = xgb_r1.predict(X_test[cols])
mse=mean_squared_error(np.rint(xgb_r1_y3), y_test)
print(mse)

C:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.514271407110666


In [28]:
mean_squared_error(xgb_r1_y3, y_test)

0.4543244510373861

In [29]:
cat_r1 = catboost.CatBoostRegressor(depth=4)
cat_r1.fit(X_train[cols], y_train,verbose=0)
cat_r1_y1 = cat_r1.predict(X_test[cols])
mse=mean_squared_error(np.rint(cat_r1_y1), y_test)
print(mse)

0.5027541311967952


In [30]:
mean_squared_error(xgb_r1_y3, y_test)

0.4543244510373861

In [49]:
mean=np.stack((xgb_r1_y3,lgb_r1_y2,cat_r1_y1,cat_r1_y1)).mean(axis=0)
mean_squared_error(mean, y_test)

0.441173021688527

In [51]:
lgb_r1.fit(X_full[cols], y,eval_metric ='mean_squared_error')
xgb_r1.fit(X_full[cols], y)
cat_r1.fit(X_full[cols], y,verbose=0)

C:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [53]:
test_submit_y1 = cat_r1.predict(test_submit[cols])
test_submit_y2 = lgb_r1.predict(test_submit[cols])
test_submit_y3 =xgb_r1.predict(test_submit[cols])

In [54]:
submit=np.stack((test_submit_y3,test_submit_y2,test_submit_y1,test_submit_y1)).mean(axis=0)

In [57]:

sub_csv=pd.Series(submit,index=test_abbr.id,name="happiness")
sub_csv.to_csv('sub_csv_6.csv',index=True,header=True)

In [56]:
np.shape(test_abbr)

(2968, 41)

In [347]:
test_submit_y1 = cat_r1.predict(test_submit[cols])

In [358]:
xgb_c1 = xgb.XGBClassifier(subsample=0.8,)
xgb_c1.fit(X_train[cols], y_train)
xgb_c1_y3 = xgb_c1.predict(X_test[cols])
mean_squared_error(np.rint(xgb_c1_y3), y_test)

0.5873810716074112

In [356]:
lgb_c1 = lgb.LGBMRegressor(boosting_type='gbdt', colsample_bytree=1,learning_rate=0.05,n_estimators=1000,max_depth=4)
lgb_c1.fit(X_train[cols], y_train,eval_metric ='mean_squared_error')
lgb_c1_y2 = lgb_c1.predict(X_test[cols],)
mean_squared_error(np.rint(lgb_r1_y2), y_test)

0.5162744116174262

In [486]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=42,shuffle=True)
parameters ={'max_depth':[2,3,4,5,6],'num_leaves':[10,20]}
lgb_cv=lgb.LGBMRegressor()
grid_xgb=GridSearchCV(estimator=lgb_cv,param_grid=parameters,cv=kf,scoring='neg_mean_squared_error')
grid_xgb.fit(X_full,y_full)
grid_xgb.best_params_
#{'max_depth': 4, 'min_child_weight': 1}
RMSE=np.sqrt(-grid_xgb.best_score_)

In [523]:
kf = KFold(n_splits=5, random_state=42,shuffle=True)
parameters ={'depth':[2,3,4,5,6],}
cat_cv=catboost.CatBoostRegressor(verbose=0)
grid_xgb=GridSearchCV(estimator=cat_cv,param_grid=parameters,cv=kf,scoring='neg_mean_squared_error')
grid_xgb.fit(X_full,y_full)
grid_xgb.best_params_

{'depth': 5}

In [524]:
np.sqrt(-grid_xgb.best_score_)

0.6772063693946946